In [ ]:
# coding: utf-8

from selenium import webdriver 
from bs4 import BeautifulSoup 
import requests 
from selenium.webdriver.common.desired_capabilities import  DesiredCapabilities 
import time 
from selenium.webdriver.common.keys import Keys 
import datetime as dt 
import pandas as pd

driver = webdriver.Chrome('C:/Program Files (x86)/Google/Chrome/Application/chromedriver.exe')

startdate=dt.date(year=2019,month=7,day=1) #start_date
untildate=dt.date(year=2019,month=7,day=2) # start_date +1 
enddate=dt.date(year=2019,month=7,day=2) # end_date



query="genome_sequencing" 
totaltweets=[] 
totalfreq=[]  

while not enddate==startdate: 
    url='https://twitter.com/search?q='+query+'%20since%3A'+str(startdate)+'%20until%3A'+str(untildate)+'&amp;amp;amp;amp;amp;amp;lang=eg' 
    driver.get(url) 
    html = driver.page_source 
    soup=BeautifulSoup(html,'html.parser') 
     
    lastHeight = driver.execute_script("return document.body.scrollHeight") 
    while True: 
         
        dailyfreq={'date':startdate} 

        wordfreq=0 
        tweets=soup.find_all("p", {"class": "TweetTextSize"}) 
                     
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
        time.sleep(1) 
         
        newHeight = driver.execute_script("return document.body.scrollHeight") 
        

        if newHeight != lastHeight: 
            html = driver.page_source 
            soup=BeautifulSoup(html,'html.parser') 
            tweets=soup.find_all("p", {"class": "TweetTextSize"}) 
            wordfreq=len(tweets) 
        else: 
            totalfreq.append(dailyfreq) 
        #    dailyfreq['Frequency']=wordfreq 
            wordfreq=0 
            
         
            
            startdate=untildate 
            untildate+=dt.timedelta(days=1) 
            dailyfreq={} 
            totaltweets.append(tweets) 
            break 

        lastHeight = newHeight

bb=""        
bb =str(totalfreq[0])
print("33")
print(bb)
print(totalfreq[0])


df = pd.DataFrame(columns=['id','message','Date'])

number=1 
for i in range(len(totaltweets)): 
    for j in range(len(totaltweets[i])): 
        df = df.append({'id': number,'message':(totaltweets[i][j]).text,'Date': str(totalfreq[i]).replace("'date': datetime.date","")}, ignore_index=True) 
        number = number+1
        

df

writer = pd.ExcelWriter('filab_tw.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()
